<a href="https://colab.research.google.com/github/HWANG-HOE-SUN/Univ.Project/blob/main/%EC%A7%84%ED%95%B4Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **고장코드 有 변환 후**

### 데이터 불러오기

In [1]:
import os
from scipy import io
import numpy as np
import pandas as pd
import copy
import matplotlib.pyplot as plt
import re # 텍스트 검색(정규표현)
import struct
%matplotlib inline

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
path_dir = '/content/gdrive/MyDrive/DATA/고장코드 有_변환 후/27.진해_6033_200115_P1F0166,0066'
file_list = os.listdir(path_dir)

In [ ]:
for file in file_list:
  print(file)

진해_6033_20200114061311.mat
진해_6033_20200114065945.mat
진해_6033_20200114092858.mat
진해_6033_20200114115522.mat
진해_6033_20200114142250.mat
진해_6033_20200114164902.mat
진해_6033_20200114191610.mat
진해_6033_20200114211913.mat
진해_6033_20200115044101.mat
분할


In [ ]:
matfile14=[]
dist = path_dir+'/'+file_list[7] # file_list 1~6까진 분할파일에서 얻어야 함 (file size때문)
matfile14.append(io.loadmat(dist)) # 14일 

In [ ]:
feature_name = []
for file in matfile14:
  feature_name.append(list(file.keys()))
for i in feature_name:
  print(len(i))

1142


In [ ]:
test = pd.DataFrame(matfile14)

In [ ]:
col_len=[]
for i in range(len(test.T)):
  col_len.append(len(test.iloc[0,i]))

In [ ]:
np.argmax(col_len) # 최대 시간 길이를 가진 칼럼의 위치

47

In [ ]:
test.iloc[:,np.argmax(col_len)].iloc[0]

array([[240.1402 ,   0.     ],
       [240.1497 ,   0.     ],
       [240.1597 ,   0.     ],
       ...,
       [301.4632 ,   0.     ],
       [301.4732 ,   0.     ],
       [301.48315,   0.     ]])

In [ ]:
test.iloc[0,].iloc[0]

array([[2.40131550e+02, 2.01211479e-01],
       [2.40141650e+02, 2.01211479e-01],
       [2.40151550e+02, 2.01211479e-01],
       ...,
       [3.01458350e+02, 8.98832970e-02],
       [3.01468350e+02, 8.98832970e-02],
       [3.01478350e+02, 8.98832970e-02]])

# 데이터 통합

### 채널 길이 확인

In [ ]:
excel_path = '/content/gdrive/MyDrive/학부연구생/겹치는채널.xlsx'
useful_excel1 = pd.read_excel(excel_path,sheet_name=0)
useful_excel2 = pd.read_excel(excel_path,sheet_name=1)
useful_excel3 = pd.read_excel(excel_path,sheet_name=2)
useful_excel4 = pd.read_excel(excel_path,sheet_name=3)
useful_excel5 = pd.read_excel(excel_path,sheet_name=4)
useful_excel6 = pd.read_excel(excel_path,sheet_name=5)

In [ ]:
sig1 = useful_excel1['SignalName'][:-1]
useful_excel1['SignalName'][:-1] # 맨 뒤 제외 , 휠모터 고장 연관 채널

0             BM1__AxuInverter__Inv_Battery_V
1           BM1__AxuInverter__INV_Fault_Check
2              BM1__AxuInverter__Inv_IptVol_V
3              BM1__AxuInverter__INV_Power_kW
4         BM1__AxuInverter__Mot_ActRotSpd_rpm
                        ...                  
122    CCAN__VCU_EFD__CR_VCU_EVCoolantAuxTemp
123    CCAN__VCU_EFD__CR_VCU_EVCoolantMotTemp
124    CCAN__VCU_EFD__CR_VCU_EVCoolantRadTemp
125             CCAN__VCU_M2__CR_VCU_ActDmdTq
126          CCAN__VCU_M2__CR_VCU_DriverDmdTq
Name: SignalName, Length: 127, dtype: object

In [ ]:
sig2 = useful_excel2['SignalName']
print(sig2.head(),'\n AuxInv 길이:',len(sig2))  # AuxInv

0        BM1__AxuInverter__Inv_Battery_V
1      BM1__AxuInverter__INV_Fault_Check
2         BM1__AxuInverter__Inv_IptVol_V
3         BM1__AxuInverter__INV_Power_kW
4    BM1__AxuInverter__Mot_ActRotSpd_rpm
Name: SignalName, dtype: object 
 AuxInv 길이: 25


In [ ]:
sig3 = useful_excel3['SignalName']
print(sig3.head(),'\n BMS 길이:',len(sig3))  # BMS

0            BM1__BMS10__CR_Bms_BulkSOC
1       BM1__BMS10__CR_Bms_EvEnergyCapa
2    BM1__BMS10__CR_Bms_EVTargetCurrent
3       BM1__BMS10__CR_Bms_EVTargetVolt
4            BM1__BMS10__CR_Bms_FullSOC
Name: SignalName, dtype: object 
 BMS 길이: 339


In [ ]:
sig4 = useful_excel4['SignalName']
print(sig4.head(),'\n EBS 길이:',len(sig4))  # EBS

0       CCAN__CVW__GrossCombinationVehicleWeight
1                CCAN__CVW__PoweredVehicleWeight
2    CCAN__DM1_EBS__AmberWarningLampStatus_EBS_C
3                      CCAN__DM1_EBS__DTC1_EBS_C
4                      CCAN__DM1_EBS__DTC2_EBS_C
Name: SignalName, dtype: object 
 EBS 길이: 143


In [ ]:
sig5 = useful_excel5['SignalName']
print(sig5.head(),'\n MCU 길이:',len(sig5))  # MCU

0    BM1__DM1_MCU_L__AmberWarningLampStatus_MCU_L_B
1                      BM1__DM1_MCU_L__DTC1_MCU_L_B
2                      BM1__DM1_MCU_L__DTC2_MCU_L_B
3                      BM1__DM1_MCU_L__DTC3_MCU_L_B
4                      BM1__DM1_MCU_L__DTC4_MCU_L_B
Name: SignalName, dtype: object 
 MCU 길이: 66


In [ ]:
sig6 = useful_excel6['SignalName']
print(sig6.head(),'\n VCU 길이:',len(sig6))  # VCU

0    BM1__DM1_VCU__AmberWarningLampStatus_VCU_B
1                      BM1__DM1_VCU__DTC1_VCU_B
2                      BM1__DM1_VCU__DTC2_VCU_B
3                      BM1__DM1_VCU__DTC3_VCU_B
4                      BM1__DM1_VCU__DTC4_VCU_B
Name: SignalName, dtype: object 
 VCU 길이: 157


In [ ]:
print(len(sig1)+len(sig2)+len(sig3)+len(sig4)+len(sig5)+len(sig6))
print(len(set(sig1)|set(sig2)|set(sig3)|set(sig4)|set(sig5)|set(sig6)))
set(sig1)&set(sig2)&set(sig3)&set(sig4)&set(sig5)&set(sig6)

857
730


set()

In [ ]:
# 휠모터 고장 연관 채널(sig1) + 다른 sheet 변수명(sig2~sig6) 교집합
print('1:',set(sig1)&set(sig2)) 
print('2:',set(sig1)&set(sig3))
print('3:',set(sig1)&set(sig4))
print('4:',set(sig1)&set(sig5))
print('5:',set(sig1)&set(sig6))

1: {'BM1__AxuInverter__Mot_TKO_Sensor', 'BM1__AxuInverter__Inv_Battery_V', 'BM1__AxuInverter__Mot_APU_Sensor', 'BM1__AxuInverter__Mot_ActRotSpd_rpm', 'BM1__AxuInverter__INV_Power_kW', 'BM1__AxuInverter__MotInv_Mode_Status', 'BM1__AxuInverter__MotInvRdy', 'BM1__AxuInverter__Mot_APU_Stats', 'BM1__AxuInverter__MotInv_Temp_C', 'BM1__AxuInverter__INV_Fault_Check', 'BM1__AxuInverter__Inv_IptVol_V', 'BM1__AxuInverter__Mot_PRE_Sensor', 'BM1__AxuInverter__Mot_CtrMod'}
2: {'BM1__BMS10__CR_Bms_BulkSOC', 'BM1__BMS10__CR_Bms_FullSOC', 'BM1__BMS10__CR_Bms_EVTargetCurrent', 'BM1__BMS10__CR_Bms_EVTargetVolt', 'BM1__BMS11__CR_Bms_BatTotV_V', 'BM1__BMS10__CR_Bms_EvEnergyCapa'}
3: {'CCAN__EBC1__EngRetarderSelection', 'CCAN__EBC1__EBSBrakeSwitch', 'CCAN__EBC1__BrakePedalPos', 'CCAN__HRW__RearAxleRightWheelSpeed', 'CCAN__HRW__RearAxleLeftWheelSpeed'}
4: {'BM1__MCU_R1__MotoringTorqueUpperLimit', 'BM1__DM1_MCU_L__RedStopLampState_MCU_L_B', 'BM1__MCU_L2__ActualMotorRotationSpeed', 'BM1__DM1_MCU_L__FlashMalfun

In [ ]:
print('1:',len(set(sig1)&set(sig2)))
print('2:',len(set(sig1)&set(sig3)))
print('3:',len(set(sig1)&set(sig4)))
print('4:',len(set(sig1)&set(sig5)))
print('5:',len(set(sig1)&set(sig6)))

1: 13
2: 6
3: 5
4: 66
5: 37


In [ ]:
13+6+5+66+37 
# 휠모터고장연관채널 sheet는 각 sheet별로 고장에 제일 연관되어 보이는 것들만 뽑아 놓은 것

127

### 해당 채널 탐색

In [ ]:
final_set = set(sig1)|set(sig2)|set(sig3)|set(sig4)|set(sig5)|set(sig6)
final_name = list(final_set)
final_name[:5] # 선정된 채널 이름들

['BM1__BMS_M1_CellVol__Cell_Vol3_Index',
 'BM1__BMS_S1_2__CR_Bms_BatI_A',
 'BM1__BMS4__CF_BMS_FltForCCM',
 'CCAN__RQST_VCU__CR_VCU_ReqPGN',
 'BM1__BMS9__CF_Bms_RlyWeldDetectionReq']

* 15일 mat file 탐색

In [ ]:
set(test.columns)&final_set # 15일 파일중 해당하는 채널은 1개

In [ ]:
test['CCAN__XBR1__CR_VCU_XBRUrgency'].iloc[0][0][0]

240.1399

In [ ]:
test[test.columns[0]][0][0]

array([2.40131550e+02, 2.01211479e-01])

In [ ]:
test[test.columns[0]][0]

array([[2.40131550e+02, 2.01211479e-01],
       [2.40141650e+02, 2.01211479e-01],
       [2.40151550e+02, 2.01211479e-01],
       ...,
       [3.01458350e+02, 8.98832970e-02],
       [3.01468350e+02, 8.98832970e-02],
       [3.01478350e+02, 8.98832970e-02]])

In [ ]:
test.iloc[:,np.argmin(col_len)][0]

array([], shape=(0, 2), dtype=float64)

In [ ]:
test.iloc[:,np.argmax(col_len)][0]

array([[240.1402 ,   0.     ],
       [240.1497 ,   0.     ],
       [240.1597 ,   0.     ],
       ...,
       [301.4632 ,   0.     ],
       [301.4732 ,   0.     ],
       [301.48315,   0.     ]])

In [ ]:
test['CCAN__VDC2_EBS__SteerWheelAngleSensorType'].iloc[0]

array([[240.13155,   3.     ],
       [240.14165,   3.     ],
       [240.15155,   3.     ],
       ...,
       [301.45835,   3.     ],
       [301.46835,   3.     ],
       [301.47835,   3.     ]])

* 14일 mat file 탐색

In [ ]:
def main_search(n):
  path_dir = '/content/gdrive/MyDrive/DATA/고장코드 有_변환 후/45.동남_1643_200928_P1F2086'
  file_list = os.listdir(path_dir)
  matfile14=[]
  date = file_list[n]
  dist = path_dir+'/'+date # file_list 1~6까진 분할파일에서 얻어야 함 (file size때문)
  matfile14.append(io.loadmat(dist)) # 14일 21 19 13
  feature_name = [] 
  global main_test
  main_test = pd.DataFrame(matfile14)
  col_len=[]
  for i in range(len(main_test.T)):
    col_len.append(len(main_test.iloc[0,i]))
  global max_timelen_file
  max_timelen_file = []  
  max_timelen_file.append(main_test.iloc[:,np.argmax(col_len)].iloc[0]) # 해당 일자 data중 최대 길이 시간 길이인 채널 따로 저장
  selected_chan = set(main_test.columns)&final_set # 해당 파일중 선정된 채널에 해당하는 것 (column 교집합)
  print(dist) # 파일 주소 출력
  print(len(pd.DataFrame(matfile14).T)) # 데이터프레임 길이(채널 갯수)
  print(date)   # 날짜 표시
  print(max(col_len)) # 채널 중 최대 시간 길이
  print(len(selected_chan)) # 해당 채널의 총 길이
  global selected_df
  selected_df = main_test[selected_chan]

In [ ]:
def wheel_search(n):
  path_dir = '/content/gdrive/MyDrive/DATA/고장코드 有_변환 후/27.진해_6033_200115_P1F0166,0066'
  file_list = os.listdir(path_dir)
  matfile14=[]
  date = file_list[n]
  dist = path_dir+'/'+date # file_list 1~6까진 분할파일에서 얻어야 함 (file size때문)
  matfile14.append(io.loadmat(dist)) # 14일 21 19 13
  global wheel_test
  wheel_test = pd.DataFrame(matfile14)
  col_len=[]
  for i in range(len(wheel_test.T)):
    col_len.append(len(wheel_test.iloc[0,i]))
  global max_timelen_file2
  max_timelen_file2 = []  
  max_timelen_file2.append(wheel_test.iloc[:,np.argmax(col_len)].iloc[0]) # 해당 일자 data중 최대 길이 시간 길이인 채널 따로 저장
  selected_chan = set(wheel_test.columns)&set(sig1) # 해당 파일중 wheel모터 고장 채널과의 교집합 
  print(dist) # 파일 주소 출력
  print(len(pd.DataFrame(matfile14).T)) # 데이터프레임 길이(채널 갯수)
  print(date)   # 날짜 표시
  print(max(col_len)) # 채널 중 최대 시간 길이
  print(len(selected_chan)) # 해당 채널의 총 길이
  global wheel_df
  wheel_df = wheel_test[selected_chan].T
  return wheel_df

In [ ]:
# hi_1 = set(wheel_search(0).T.columns)
# hi_2 = set(wheel_search(11).T.columns)
# hi_1&hi_2

/content/gdrive/MyDrive/DATA/고장코드 有_변환 후/27.진해_6033_200115_P1F0166,0066/분할/진해_6033_20200114191610_1.mat
631
진해_6033_20200114191610_1.mat
709706
43


In [ ]:
wheel_search(8)

/content/gdrive/MyDrive/DATA/고장코드 有_변환 후/27.진해_6033_200115_P1F0166,0066/진해_6033_20200115044101.mat
609
진해_6033_20200115044101.mat
1484
32


,0
CCAN__EMRC1__CR_Vcu_Em1ActMaxAvailRtdTq,"[[240.45275, 0.0], [240.46365, 0.0], [240.4727..."
CCAN__EMRC1__CR_Vcu_Em1ActPerTorq,"[[240.45275, 0.0], [240.46365, 0.0], [240.4727..."
CCAN__EEC2__CR_VCU_AccelPedPos,"[[240.46275, 0.0], [240.48265, 0.0], [240.5027..."
CCAN__EMRC1__CR_Vcu_Em1RtdPerTorq,"[[240.45275, 0.0], [240.46365, 0.0], [240.4727..."
CCAN__ETC2__CR_VCU_SelGear,"[[240.54365, 0.0], [240.64365, 0.0], [240.7436..."
CCAN__VCU_CLUINF1__CF_VCU_MotorOverTemp,"[[240.5451, 0.0], [240.6451, 0.0]]"
BM1__BMS10__CR_Bms_EVTargetVolt,"[[241.20325, 0.0], [241.30325, 0.0], [241.4032..."
CCAN__EMRC1__CR_Vcu_Em1RtdSelect,"[[240.45275, 0.0], [240.46365, 0.0], [240.4727..."
BM1__BMS10__CR_Bms_EvEnergyCapa,"[[241.20325, 64000.0], [241.30325, 64000.0], [..."
CCAN__VCU_M2__CR_VCU_DriverDmdTq,"[[240.45305, 0.0], [240.46395, 0.0], [240.4730..."


In [ ]:
wheel_search(2).to_excel(excel_writer='Daejin_wheel.xlsx') # 최소갯수 추출

/content/gdrive/MyDrive/DATA/고장코드 有_변환 후/41.대진_1872_200803_P1F2806/대진_1872_20200803092814.mat
521
대진_1872_20200803092814.mat
63
6


In [ ]:
main_search(7) # 함수 주소의 해당 폴더 n번째 파일

/content/gdrive/MyDrive/DATA/고장코드 有_변환 후/45.동남_1643_200928_P1F2086/동남_1643_20200928164616.mat
1184
동남_1643_20200928164616.mat
23343
668


In [ ]:
max_timelen_file[0][0],max_timelen_file[0][-1]

(array([240.11185,   0.     ]), array([301.7644,   0.    ]))

In [ ]:
max_timelen_file[0]

array([[240.02275,   0.     ],
       [240.0328 ,   0.     ],
       [240.04285,   0.     ],
       ...,
       [507.67895,   0.     ],
       [507.68895,   0.     ],
       [507.699  ,   0.     ]])

In [ ]:
# pd.DataFrame(main_test.columns).to_excel(excel_writer='sample.xlsx')

In [ ]:
selected_df

,CCAN__TSC1_290B__EngineRequestedTorqueHiRes,BM1__BMS_M1_TEMP__BMS_TEMP_2,CCAN__VCU_M2__CR_VCU_DriverDmdTq,BM1__BMS1_M1__CR_Bms_BatChaLmt_W,BM1__BMS9__CF_Bms_BatHeatStat,BM1__BMS_M1_TEMP__BMS_TEMP_8,CCAN__VDC1_EBS__YCEngCtrlActive,BM1__CMM6_BM2__CF_CMM_DCChargingStat,CCAN__VCU_CLUINF1__CF_VCU_StartingPro,BM1__BMS_S2_3__CR_Bms_CoolStatus,BM1__BMS_S2_3__CF_Serge_Level,BM1__BMS_S2_1__CF_Bms_Wrn,BM1__VCU_BMS__CF_VCU_VehChgEn,BM1__MCU_R2__SeviceLampRequestofMCU,BM1__BMS5_M1__CR_Bms_Outlet,CCAN__EMC1_EM1__EM1_MessagChecksum,BM1__BMS_S2_1__Relay_ON_Status_Acon,BM1__BMS_S1_1__CR_Selection_BMU,BM1__BMS_S1_3__CF_Slave_LCRR_ON,CCAN__EBC1__TrctrMntdTrilerABSWarningSignal,BM1__BMS13__CR_Bms_PwmDutyCmd,BM1__VCU_MCU_L1__CR_VCU_ReqMGTorq_L,BM1__BMS3_M1__CF_Bms_Diag_Group,BM1__MCU_L3__MCUWarningCode,BM1__BMS_S2_1__CF_BMU_Slave_Decision,CCAN__EBC4_BRK__BrkLnngRmningRearAxle2LeftWheel,BM1__VCU_BMS__CF_VCU_IG3_RlyOn,BM1__BMS_M1__CF_BMU_Master_Ready,BM1__BMS_M2__CF_Master2_LCRR_Permit,BM1__DM1_MCU_R__DTC4_MCU_R_B,BM1__MCU_R2__MotorControlState,CCAN__VCU_CLUINF3__CR_VCU_V_BAT,BM1__BMS_S1_TEMP__BMS_TEMP_4,BM1__BMS_S2_3__CF_Slave_LCRR_Permit,BM1__BMS9__CF_Bms_Fail_VoltRange,BM1__BMS_S2_2__CR_Bms_BatMaxTemp_C,BM1__BMS_S2_TEMP__BMS_TEMP_7,CCAN__TSC1_000B__TransmissionRate,BM1__BMS_S1_1__CR_BMU_Type_BSW,BM1__BMS_M1_TEMP__BMS_TEMP_5,...,BM1__BMS8__CR_Bms_CCMMainsVol,BM1__AxuInverter__INV_Fault_Check,BM1__BMS_M2_CellVol__Cell_Vol4,BM1__VCU_MCU_L1__CR_VCU_MGRefTorq_L,BM1__BMS_M1_CellVol__Cell_Vol4,CCAN__EBC4_BRK__BrkLnngRmningRearAxle1LeftWheel,CCAN__EEC2_EM1__CR_Vcu_EngLoadatSpd_Per,BM1__BMS_S2_1__CF_Relay_ON_Status,BM1__BMS_S1_TEMP__BMS_TEMP_3,CCAN__EEC1__CR_VCU_EngineSpd,BM1__BMS_M2_CellVol__Cell_Vol3,CCAN__TSC1_290B__EngRqedSpeed_SpeedLimit,CCAN__TSC1_0F0B__EngineRequestedTorqueHiRes,CCAN__TSC1_100B__OverrideCtrlModePriority,BM1__BMS_S1_3__CR_CoolFANDuty,BM1__CMM6_BM2__CR_CMM_ACT_VOLTAGE,BM1__VCU_BMS__CF_VCU_Rdy_BMS,CCAN__DM1_EBS__ProtectLampStatus_EBS_C,BM1__BMS_M2_2__CR_Bms_BatI_A,CCAN__EBC1__ABSFullyOperational,BM1__VCU_MCU_L1__CR_VCU_MsgChecksum_L,CCAN__EBC4_BRK__BrkLnngRmnngRearAxle3RightWheel,BM1__DM1_BMU_M1__FlashAmberWarningLamp_BMU_M1_B,CCAN__EBC5__BrakeTempWarning,CCAN__RQST__CR_VCU_LSBReqPGN,BM1__BMS_ISOL_M1__CF_Status,BM1__DM1_AuxInv__DTC2_AuxInv_B,CCAN__EBC3__BrkAppPrssHghRngRrAxl2RghtWheel,BM1__BMS_S2_1__CF_Serge_Level,CCAN__DM1_VCU__FlashProtectLamp_VCU_B,BM1__VCU_MCU_R2__CR_VCU_GenTorqLimit_R,BM1__DM1_VCU__MalfunctionIndicatorLampStatus_VCU_B,BM1__MCU_R1__InverterInputPower,BM1__BMS_S1_1__Relay_ON_Status_Acon,CCAN__EMC1_EM1__EM1_Ovr_Ctrl_Mode_Priority_Brk,CCAN__XBR1__CR_VCU_XBRUrgency,CCAN__XBR1__CR_VCU_XBREbiMode,BM1__BMS_M2_CellVol__Cell_Vol4_Bal,BM1__BMS_S1_2__CR_Bms_BatMinCel_V,BM1__BMS9__CF_Bms_Fail_Temp
0,"[[240.0775, 1.875], [240.2775, 1.875], [240.47...","[[241.07805, 0.0], [241.17805, 0.0], [241.278,...","[[240.05115, 0.0], [240.06125, 0.0], [240.0710...","[[241.0517, 500000.0], [241.1517, 500000.0], [...","[[241.0482, 0.0], [241.1482, 0.0], [241.2482, ...","[[241.07805, 22.0], [241.17805, 0.0], [241.278...","[[240.0575, 0.0], [240.1575, 0.0], [240.2575, ...","[[241.05535, 0.0], [241.15535, 0.0], [241.2553...","[[240.1131, 0.0], [240.21305, 0.0], [240.3143,...","[[241.0643, 2.0], [241.16435, 2.0], [241.2643,...","[[241.0643, 0.0], [241.16435, 0.0], [241.2643,...","[[241.0654, 0.0], [241.16545, 0.0], [241.2654,...","[[241.01135, 1.0], [241.11135, 1.0], [241.2113...","[[240.99235, 0.0], [241.0024, 0.0], [241.01235...","[[241.05505, 23.0], [241.15505, 23.0], [241.25...","[[240.1175, 6.0], [240.2175, 1.0], [240.3175, ...","[[241.0654, 0.0], [241.16545, 0.0], [241.2654,...","[[241.066, 1.0], [241.166, 2.0], [241.266, 3.0...","[[241.06485, 0.0], [241.1649, 0.0], [241.26485...","[[240.0556, 3.0], [240.1556, 3.0], [240.2556, ...","[[241.0492, 0.0], [241.1492, 0.0], [241.2492, ...","[[240.9891, 0.0], [240.9991, 0.0], [241.0091, ...","[[241.0539, 0.0], [241.1539, 0.0], [241.2539, ...","[[241.00135, 0.0], [241.10105, 0.0], [241.2011...","

In [ ]:
print(len(main_test['CCAN__XBR1__CR_VCU_XBRUrgency'].iloc[0])) # 해당 채널 데이터 길이
main_test['CCAN__XBR1__CR_VCU_XBRUrgency'].iloc[0]

223091


array([[ 240.0497 ,    0.     ],
       [ 240.06965,    0.     ],
       [ 240.08965,    0.     ],
       ...,
       [4702.0492 ,    0.     ],
       [4702.06925,    0.     ],
       [4702.0892 ,    0.     ]])

In [ ]:
pd.DataFrame(main_test['CCAN__XBR1__CR_VCU_XBRUrgency'].iloc[0])

In [ ]:
np.unique(pd.DataFrame(main_test['CCAN__XBR1__CR_VCU_XBRUrgency'].iloc[0])[1])

array([0.])

### 무고장

In [ ]:
def main_mu(n):
  path_dir = '/content/gdrive/MyDrive/DATA/무고장데이터_5일간/27.진해_6033_200110_200114_5일간'
  file_list = os.listdir(path_dir)
  matfile=[]
  date = file_list[n]
  dist = path_dir+'/'+date # file_list 1~6까진 분할파일에서 얻어야 함 (file size때문)
  matfile.append(io.loadmat(dist)) #무고장
  feature_name = [] 
  global main_mu
  main_df = pd.DataFrame(matfile)
  print(dist) # 파일 주소 출력
  print(date)   # 날짜 표시
  return main_df

In [ ]:
mu_2 = main_mu(1)
mu_2.head()

/content/gdrive/MyDrive/DATA/무고장데이터_5일간/27.진해_6033_200110_200114_5일간/진해_6033_20200111053415.mat
진해_6033_20200111053415.mat


,CCAN__HRW__FrontAxl,CCAN__HRW__RearAxle,CCAN__SAS__Steering,CCAN__SAS__MessageC,CCAN__EMC1_EM1__EM1,CCAN__ETC1__CR_VCU_,CCAN__EEC1__CR_VCU_,CCAN__CCVS_VCU__CF_,CCAN__CCVS_VCU__CR_,CCAN__EMRC1__CR_Vcu,CCAN__VCU_M2__CR_VC,CCAN__VDC2_EBS__Ste,CCAN__VDC2_EBS__Yaw,CCAN__VDC2_EBS__Lat,CCAN__VDC2_EBS__Lon,CCAN__XBR1__CR_VCU_,CCAN__EEC2__CR_VCU_,CCAN__EEC2_EM1__CR_,CCAN__DODmsg01__C_D,CCAN__DODmsg01__C_R,CCAN__DODmsg01__C_F,CCAN__DODmsg01__C_M,CCAN__DODmsg01__C_P,CCAN__CLUmsg02__Ste,CCAN__CLUmsg02__Kin,CCAN__CLUmsg02__Ind,CCAN__CLUmsg02__Dis,CCAN__CLU_TestMsg02,CCAN__CLU_TestMsg03,CCAN__AEBS2__AEBS_S,CCAN__AEBS2__AEBS_D,CCAN__AEBS2__AEBS_C,CCAN__TSC1_000B__En,CCAN__TSC1_000B__Ov,CCAN__TSC1_000B__Tr,CCAN__TSC1_000B__Co,CCAN__TSC1_000B__Me,CCAN__EBC2__FrontAx,CCAN__EBC2__Relativ,CCAN__EBC2__Rlative,...,BM1__DM1_LDC__RedSt,BM1__DM1_LDC__Malfu,BM1__DM1_LDC__Flash,BM1__DM1_LDC__DTC1_,BM1__DM1_LDC__DTC2_,BM1__DM1_LDC__DTC3_,BM1__DM1_LDC__DTC4_,BM1__DM1_AuxInv__Pr,BM1__DM1_AuxInv__Am,BM1__DM1_AuxInv__Re,BM1__DM1_AuxInv__Ma,BM1__DM1_AuxInv__Fl,BM1__DM1_AuxInv__DT,BM1__DM1_VCU__Prote,BM1__DM1_VCU__Amber,BM1__DM1_VCU__RedSt,BM1__DM1_VCU__Malfu,BM1__DM1_VCU__Flash,BM1__DM1_VCU__DTC1_,BM1__DM1_VCU__DTC2_,BM1__DM1_VCU__DTC3_,BM1__DM1_VCU__DTC4_,CCAN__RQST__CR_VCU_,CCAN__CVW__PoweredV,CCAN__CVW__GrossCom,BM1__DM1_BMU_M1__Pr,BM1__DM1_BMU_M1__Am,BM1__DM1_BMU_M1__Re,BM1__DM1_BMU_M1__Ma,BM1__DM1_BMU_M1__Fl,BM1__DM1_BMU_M1__DT,CCAN__TPCM_VCU__Con,CCAN__TPCM_VCU__Tot,CCAN__TPCM_VCU__PGN,CCAN__TPDT_VCU__Seq,CCAN__TPDT_VCU__Pac,BM1__BMS_M2_CMMGTX_,BM1__CMM4_BM2__CR_C,BM1__CMM4_BM2__CF_C,BM1__CMM4_BM2__CF_E
0,"[[240.369, 0.0], [240.3892, 0.0], [240.40905, ...","[[240.369, 0.0], [240.3892, 0.0], [240.40905, ...","[[240.37175, 1.0], [240.38175, 1.0], [240.3918...","[[240.37175, 13.0], [240.38175, 14.0], [240.39...","[[240.37305, 1.0], [240.47305, 0.0], [240.5730...","[[240.37485, 0.0], [240.3848, 0.0], [240.3948,...","[[240.3751, 1000.0], [240.3851, 1000.0], [240....","[[240.3754, 1.0], [240.3863, 1.0], [240.3954, ...","[[240.3754, 5.0], [240.3863, 5.0], [240.3954, ...","[[240.3757, 0.0], [240.38655, 0.0], [240.3957,...","[[240.376, 77.0], [240.38685, 77.0], [240.3959...","[[240.377, 3.0], [240.3871, 3.0], [240.397, 3....","[[240.377, -0.0013088599999999673], [240.3871,...","[[240.377, -0.039547073999997906], [240.3871, ...","[[240.377, 0.0], [240.3871, 0.0], [240.397, 0....","[[240.3854, 10.0], [240.4054, 11.0], [240.4254...","[[240.3857, 0.0], [240.4057, 0.0], [240.4257, ...","[[240.386, 0.0], [240.406, 0.0], [240.426, 0.0...","[[240.3889, 0.0], [240.4389, 0.0], [240.4889, ...","[[240.3889, 0.0], [240.4389, 0.0], [240.4889, ...","[[240.3889, 3.0], [240.4389, 3.0], [240.4889, ...","[[240.3889, 3.0], [240.4389, 3.0], [240.4889, ...","[[240.3889, 0.0], [240.4389, 0.0], [240.4889, ...","[[240.39205, 1.0], [240.44235, 1.0], [240.4918...","[[240.39205, 0.0], [240.44235, 0.0], [240.4918...","[[240.39205, 0.0], [240.44235, 0.0], [240.4918...","[[240.39205, 0.0], [240.44235, 0.0], [240.4918...","[[240.39235, 0.0], [240.44265, 0.0], [240.4923...","[[240.39265, 0.0], [240.44295, 0.0], [240.4926...","[[240.4001, 1.0], [240.45015, 1.0], [240.50005...","[[240.4001, 255.0], [240.45015, 255.0], [240.5...","[[240.4001, 9.0], [240.45015, 10.0], [240.5000...","[[240.4071, 1.875], [240.5071, 1.875], [240.60...","[[240.4071, 3.0], [240.5071, 3.0], [240.60715,...","[[240.4071, 7.0], [240.5071, 7.0], [240.60715,...","[[240.4071, 31.0], [240.5071, 31.0], [240.6071...","[[240.4071, 3.0], [240.5071, 4.0], [240.60715,...","[[240.40965, 0.0], [240.45905, 0.0], [240.5095...","[[240.40965, 8.125], [240.45905, 8.125], [240....","[[240.40965, 8.125], [240.45905, 8.125], [240....",...,"[[241.69575, 0.0], [242.6958, 0.0], [243.6961,...","[[241.69575, 0.0], [242.6958, 0.0], [243.6961,...","[[241.69575, 3.0], [242.6958, 3.0], [243.6961,...","[[241.69575, 4278190080.0], [242.6958, 4278190...",[],[],[],"[[241.82155, 0.0], [242.87145, 0.0], [243.9217...","[[241.82155, 0.0], [242.87145, 0.0], [2

In [ ]:
set(mu_2.columns)&final_set

set()

### dd

In [ ]:
useful_index = []
for idx in range(0, len(matfile1)): # feature가 각각 142,1142,1142,1142 서로 다른 길이의 파일 총 4개 에서 for문
    useful_idx = []  # 빈 리스트 4개 생성
    for idx2, keys in enumerate(feature_name[idx]): # 142,1142,1142,1142개의 항목... 각 idx와 key값(피처이름)
        length = show_duplicate(matfile1[idx][keys]) 
        if length > 1 :
            useful_idx.append((idx2,length, len(matfile1[idx][keys])))
    useful_index.append(useful_idx)    

In [ ]:
useful_index[0] # [0~3]까지 총 4항목 존재
# 각각의 값은 tuple형태로 저장되어있고 각 값의 의미는... 순서대로
# (1. feature가있는 위치(feature index), 2. unique한 값 갯수 , 3. 해당 feature의 전체 data수)

In [ ]:
useful_col=[]
for s in useful_index[0]:
    useful_col.append(s[0])
print(len(useful_col))

In [ ]:
pd.DataFrame(matfile1).T.iloc[useful_col] # useful한 column들만 남겨놓는다.

In [ ]:
useful_df = pd.DataFrame(matfile1).T.iloc[useful_col]

### DTC 추출

In [ ]:
import re
p = re.compile('.*DTC1.*')
dtc_set = []
for f_name in feature_name:
    dtc_index = []
    for idx, name in enumerate(f_name):
        m = p.match(name)
        if m != None:
            print(m.group())
            dtc_index.append(idx)
    dtc_set.append(dtc_index)

In [ ]:
for i in dtc_set:
    print(len(i))
    print(dtc_set[0])

In [ ]:
dtc_name = [] # 이름 list
for i in range(len(dtc_set)):
  print(i+1,'번째 파일의 dtc이름 및 index')
  for dtc in dtc_set[i]:
    print(dtc,feature_name[0][dtc])
    dtc_name.append(feature_name[0][dtc])

In [ ]:
matfile13[0]['CCAN__DM1_DOD__DTC1_DOD_B']

In [ ]:
DTC_df_1 = pd.DataFrame(matfile1)[dtc_name].T
DTC_df_1

In [ ]:
DTC_df_1[0].iloc[2]

In [ ]:
for i in range(len(DTC_df_1)): # DTC 피쳐별로 데이터 개수가 다름(수집 시간 차이?)
  print(len(DTC_df_1[0].iloc[i]))

In [ ]:
def float_to_hex(f):
    return hex(struct.unpack('<I', struct.pack('<f', f))[0])

In [ ]:
len(DTC_df_1)

In [ ]:
dtc_col = []
dtc_val = []
for i in range(len(DTC_df_1)):
  dtc_col.append(DTC_df_1.index[i]) # DTC Feature 이름
  dtc_val.append(DTC_df_1[0].iloc[i][:,1]) # value 2-d array의 i번째 인덱스의 두번째값(실제 value)

In [ ]:
pd.DataFrame(dtc_val,dtc_col).T

> NAN값 없는 범위에 대해서만 생성

In [ ]:
dtc_df = pd.DataFrame(dtc_val,dtc_col).T[:6898]

In [ ]:
dtc_df = dtc_df[:6898]

In [ ]:
dtc_hex_df = dtc_df

In [ ]:
dtc_hex_df

In [ ]:
for i in dtc_hex_df.columns:
  dtc_hex_df[i] = dtc_hex_df[i].apply(float_to_hex)
dtc_hex_df

# **고장코드 無 변환 후**

In [ ]:
path_dir = '/content/gdrive/MyDrive/DATA/무고장데이터_5일간/27.진해_6033_200110_200114_5일간'
file_list13 = os.listdir(path_dir)

In [ ]:
file_list13 = file_list[-20:-8] # 13일 
file_list13

In [ ]:
matfile13 = []
count = 0
for file in file_list13:
    dist = path_dir + '/' + file
    count+=1
    if count==1: 
      matfile13.append(io.loadmat(dist)) # 13일 첫번째 파일


In [ ]:
pd.DataFrame(matfile13).T[0]

In [ ]:
feature_name = []
for file in matfile13:
    feature_name.append(list(file.keys()))

for i in feature_name:
    print(len(i))

In [ ]:
time = []
for i in range(len(pd.DataFrame(matfile13).T[0])):
  time.append(pd.DataFrame(matfile13).T[0].iloc[i][-1][0])
print(max(time))

In [ ]:
pd.DataFrame(matfile13).T[0].iloc[2][-1][0]